In [163]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
from psycopg2.extras import execute_values

In [164]:
load_dotenv()

CODER_REDSHIFT_HOST = os.environ.get('CODER_REDSHIFT_HOST')
CODER_REDSHIFT_DB = os.environ.get('CODER_REDSHIFT_DB')
CODER_REDSHIFT_USER = os.environ.get('CODER_REDSHIFT_USER')
CODER_REDSHIFT_PASS = os.environ.get('CODER_REDSHIFT_PASS')
CODER_REDSHIFT_PORT = os.environ.get('CODER_REDSHIFT_PORT')

try:
    conn = psycopg2.connect(
        host=CODER_REDSHIFT_HOST,
        dbname=CODER_REDSHIFT_DB,
        user=CODER_REDSHIFT_USER,
        password=CODER_REDSHIFT_PASS,
        port=CODER_REDSHIFT_PORT,

    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


# SEGUNDA PARTE (Conexión y Parsing de Datos a partir de API AlphaVantage)

In [165]:
import requests

load_dotenv()
alphavantage_api_key = os.environ.get('ALPHAVANTAGE_API_KEY')

In [166]:
def get_json(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&interval=5min&apikey={alphavantage_api_key}"
    r = requests.get(url)
    return r.json()
def format_json(json, symbol):
    df = pd.DataFrame(json['Monthly Adjusted Time Series']).T
    df.rename(columns=lambda x: x[3:], inplace=True)
    # add index column
    df['date'] = df.index
    df.reset_index(drop=True, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df['open'] = pd.to_numeric(df['open'])
    df['high'] = pd.to_numeric(df['high'])
    df['low'] = pd.to_numeric(df['low'])
    df['close'] = pd.to_numeric(df['close'])
    df['adjusted close'] = pd.to_numeric(df['adjusted close'])
    # Normalize volume
    df['volume'] = pd.to_numeric(df['volume'])
    df['volume'] = round(df['volume'] / 1000000)
    df['volume'] = df['volume'].astype(int)
    df['dividend amount'] = pd.to_numeric(df['dividend amount'])
    df['symbol'] = symbol
    df['symbol'] = df['symbol'].astype(str)
    return df

In [167]:
data = get_json('AAPL')
df_aapl = format_json(data, 'AAPL')
df_aapl.head(2)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,171.220,179.85,170.82,179.80,179.80,394,0.0,2023-10-11,AAPL
1,189.485,189.98,167.62,171.21,171.21,1338,0.0,2023-09-29,AAPL


In [168]:
data = get_json('AMZN')
df_amzn = format_json(data, 'AMZN')
df_amzn.head(2)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,127.280,132.05,124.13,131.83,131.83,352,0.0,2023-10-11,AMZN
1,139.455,145.86,123.04,127.12,127.12,1121,0.0,2023-09-29,AMZN


In [169]:
data = get_json('GOOG')
df_goog = format_json(data, 'GOOG')
df_goog.head(2)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,132.155,142.22,132.065,141.70,141.70,155,0.0,2023-10-11,GOOG
1,138.430,139.93,128.190,131.85,131.85,390,0.0,2023-09-29,GOOG


In [170]:
data = get_json('MSFT')
df_msft = format_json(data, 'MSFT')
df_msft.head(2)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,316.28,332.82,311.215,332.42,332.42,165,0.0,2023-10-11,MSFT
1,331.31,340.86,309.450,315.75,315.75,417,0.0,2023-09-29,MSFT


In [171]:
data = get_json('IBM')
df_ibm = format_json(data, 'IBM')
df_ibm.head(2)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,140.04,143.4150,139.86,143.22,143.22,24,0.0,2023-10-11,IBM
1,147.26,151.9299,139.61,140.30,140.30,83,0.0,2023-09-29,IBM


In [173]:
data = get_json('PFE')
df_pfe = format_json(data, 'PFE')
df_pfe.head(2)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,32.95,34.11,32.7600,33.10,33.10,205,0.0,2023-10-11,PFE
1,35.64,36.29,31.7744,33.17,33.17,555,0.0,2023-09-29,PFE


In [174]:
# Se visualiza el tipo de dato de cada columna para la creación de la tabla
df_aapl.dtypes

open                      float64
high                      float64
low                       float64
close                     float64
adjusted close            float64
volume                      int32
dividend amount           float64
date               datetime64[ns]
symbol                     object
dtype: object

In [175]:
def cargar_en_redshift(conn, table_name, dataframe):
    dtypes = dataframe.dtypes
    cols = list(dtypes.index)
    print(cols)
    tipos = list(dtypes.values)
    type_map = {
        'float64': 'FLOAT',
        'int32': 'INT',
        'datetime64[ns]': 'TIMESTAMP',
        'object': 'VARCHAR(255)'
    }
    sql_dtypes = [type_map.get(str(dtype), 'VARCHAR(255)') for dtype in tipos]

    # Definir formato SQL VARIABLE TIPO_DATO
    column_defs = [f'"{name}" {data_type}' for name, data_type in zip(cols, sql_dtypes)]

    # Combine column definitions into the CREATE TABLE statement
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(column_defs)});
        """

    # Crear la tabla
    cur = conn.cursor()
    try:
        cur.execute(table_schema)

        # Generar los valores a insertar
        values = [tuple(x) for x in dataframe.values]

        # Definir el INSERT
        insert_sql = f"INSERT INTO {table_name} (\"open\", \"high\", \"low\", \"close\", \"adjusted close\", \"volume\", \"dividend amount\", \"date\", \"symbol\") VALUES %s"

        # Execute the transaction to insert the data
        cur.execute("BEGIN")
        execute_values(cur, insert_sql, values)
        cur.execute("COMMIT")
        print('Proceso terminado')
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback the transaction on error

def drop_table(conn, table_name):
    cur = conn.cursor()
    try:
        cur.execute(f"DROP TABLE IF EXISTS {table_name}")
        conn.commit()
        print('Proceso terminado')
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback the transaction on error


In [176]:
drop_table(conn=conn, table_name='apple_example')

Proceso terminado


In [177]:
cargar_en_redshift(conn=conn, table_name='apple_example', dataframe=df_aapl)

['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'dividend amount', 'date', 'symbol']
Proceso terminado


# TERCERA PARTE (Unión de Datos y Subida a Redshift)

In [181]:
result_df = pd.concat([df_aapl, df_amzn, df_goog, df_msft, df_ibm, df_pfe], ignore_index=True)
result_df = result_df.sort_values(by=['date'], ascending=False)
result_df = result_df.reset_index(drop=True)
result_df.head(10)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,171.220,179.8500,170.8200,179.80,179.80,394,0.0,2023-10-11,AAPL
1,32.950,34.1100,32.7600,33.10,33.10,205,0.0,2023-10-11,PFE
2,316.280,332.8200,311.2150,332.42,332.42,165,0.0,2023-10-11,MSFT
3,127.280,132.0500,124.1300,131.83,131.83,352,0.0,2023-10-11,AMZN
4,132.155,142.2200,132.0650,141.70,141.70,155,0.0,2023-10-11,GOOG
5,140.040,143.4150,139.8600,143.22,143.22,24,0.0,2023-10-11,IBM
6,138.430,139.9300,128.1900,131.85,131.85,390,0.0,2023-09-29,GOOG
7,35.640,36.2900,31.7744,33.17,33.17,555,0.0,2023-09-29,PFE
8,331.310,340.8600,309.4500,315.75,315.75,417,0.0,2023-09-29,MSFT
9,147.260,151.9299,139.6100,140.30,140.30,83,0.0,2023-09-29,IBM


In [182]:
drop_table(conn=conn, table_name='monthly_stocks_over_time')

Proceso terminado


In [183]:
cargar_en_redshift(conn=conn, table_name='monthly_stocks_over_time', dataframe=result_df)

['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'dividend amount', 'date', 'symbol']
Proceso terminado
